In [3]:
cd("/home/alir/Oceananigans.jl/")

In [220]:
using Pkg
pkg"activate ."

Activating environment at `~/Oceananigans.jl/Project.toml`


In [302]:
using Oceananigans, Oceananigans.Solvers, Statistics, OffsetArrays, FFTW; using Oceananigans: fill_halo_regions!, NoPenetrationBC;

In [293]:
@inline δx_caa(i, j, k, f) = @inbounds f[i+1, j, k] - f[i, j, k]
@inline δy_aca(i, j, k, f) = @inbounds f[i, j+1, k] - f[i, j, k]
@inline δz_aac(i, j, k, f) = @inbounds f[i, j, k+1] - f[i, j, k]

@inline ∂x_caa(i, j, k, Δx,  f) = δx_caa(i, j, k, f) / Δx
@inline ∂y_aca(i, j, k, Δy,  f) = δy_aca(i, j, k, f) / Δy
@inline ∂z_aac(i, j, k, ΔzF, f) = δz_aac(i, j, k, f) / ΔzF[k]

@inline ∂x²(i, j, k, Δx, f)       = (∂x_caa(i, j, k, Δx, f)  - ∂x_caa(i-1, j, k, Δx, f))  / Δx
@inline ∂y²(i, j, k, Δy, f)       = (∂y_aca(i, j, k, Δy, f)  - ∂y_aca(i, j-1, k, Δy, f))  / Δy
@inline ∂z²(i, j, k, ΔzF, ΔzC, f) = (∂z_aac(i, j, k, ΔzF, f) - ∂z_aac(i, j, k-1, ΔzF, f)) / ΔzC[k]

@inline div_ccc(i, j, k, Δx, Δy, ΔzF, u, v, w) = ∂x_caa(i, j, k, Δx, u) + ∂y_aca(i, j, k, Δy, v) + ∂z_aac(i, j, k, ΔzF, w)

@inline ∇²(i, j, k, Δx, Δy, ΔzF, ΔzC, f) = ∂x²(i, j, k, Δx, f) + ∂y²(i, j, k, Δy, f) + ∂z²(i, j, k, ΔzF, ΔzC, f)

∇² (generic function with 1 method)

In [294]:
function grid(zF)
    Nz = length(zF) - 1
    ΔzF = [zF[k+1] - zF[k] for k in 1:Nz]
    zC = [(zF[k] + zF[k+1]) / 2 for k in 1:Nz]
    ΔzC = [zC[k+1] - zC[k] for k in 1:Nz-1]
    return zF, zC, ΔzF, ΔzC
end

grid (generic function with 1 method)

In [295]:
arch = CPU()
Nx = Ny = 8
zF = [1, 2, 4, 7, 11, 16, 22, 29, 37]

9-element Array{Int64,1}:
  1
  2
  4
  7
 11
 16
 22
 29
 37

In [296]:
Lx, Ly, Lz = 100, 100, zF[end]
Δx, Δy = Lx/Nx, Ly/Ny

(12.5, 12.5)

In [297]:
Nz = length(zF) - 1
zF, zC, ΔzF, ΔzC = grid(zF)

# Need some halo regions.
ΔzF = OffsetArray([ΔzF[1], ΔzF...], 0:Nz)
ΔzC = [ΔzC..., ΔzC[end]]

# Useful for broadcasting z operations
ΔzC = reshape(ΔzC, (1, 1, Nz));

In [298]:
# Temporary hack: Useful for reusing fill_halo_regions! and BatchedTridiagonalSolver
# which only need Nx, Ny, Nz.
fake_grid = RegularCartesianGrid(size=(Nx, Ny, Nz), length=(Lx, Ly, Lz))

RegularCartesianGrid{Float64}
domain: x ∈ [0.0, 100.0], y ∈ [0.0, 100.0], z ∈ [0.0, -37.0]
  resolution (Nx, Ny, Nz) = (8, 8, 8)
   halo size (Hx, Hy, Hz) = (1, 1, 1)
grid spacing (Δx, Δy, Δz) = (12.5, 12.5, 4.625)

In [299]:
#####
##### Generate batched tridiagonal system coefficients and solver
#####

function λi(Nx, Δx)
    is = reshape(1:Nx, Nx, 1, 1)
    @. (2sin((is-1)*π/Nx) / Δx)^2
end

function λj(Ny, Δy)
    js = reshape(1:Ny, 1, Ny, 1)
    @. (2sin((js-1)*π/Ny) / Δy)^2
end

kx² = λi(Nx, Δx);
ky² = λj(Ny, Δy);

In [300]:
# Lower and upper diagonals are the same
ld = [1/ΔzF[k] for k in 1:Nz-1]
ud = copy(ld)

# Diagonal (different for each i,j)
@inline δ(k, ΔzF, ΔzC, kx², ky²) = - (1/ΔzF[k-1] + 1/ΔzF[k]) - ΔzC[k] * (kx² + ky²)

d = zeros(Nx, Ny, Nz)
for i in 1:Nx, j in 1:Ny
    d[i, j, :] .= [-1/ΔzF[1], [δ(k, ΔzF, ΔzC, kx²[i], ky²[j]) for k in 2:Nz]...]
end

In [335]:
# Random right hand side
Ru = CellField(Float64, arch, fake_grid)
Rv = CellField(Float64, arch, fake_grid)
Rw = CellField(Float64, arch, fake_grid)

interior(Ru) .= rand(Nx, Ny, Nz)
interior(Rv) .= rand(Nx, Ny, Nz)
interior(Rw) .= rand(Nx, Ny, Nz)

uv_bcs = HorizontallyPeriodicBCs()
w_bcs = HorizontallyPeriodicBCs(top=NoPenetrationBC(), bottom=NoPenetrationBC())

fill_halo_regions!(Ru.data, uv_bcs, arch, fake_grid)
fill_halo_regions!(Rv.data, uv_bcs, arch, fake_grid)
fill_halo_regions!(Rw.data,  w_bcs, arch, fake_grid)

R = zeros(Nx, Ny, Nz)
for i in 1:Nx, j in 1:Ny, k in 1:Nz
    R[i, j, k] = div_ccc(i, j, k, Δx, Δy, ΔzF, Ru.data, Rv.data, Rw.data)
end

# R .= R .- mean(R)  # Need RHS with zero mean

@show sum(R)

F = zeros(Nx, Ny, Nz)
F = ΔzC .* R;  # RHS needs to be multiplied by ΔzC

sum(R) = 30.748259215747773


In [336]:
#####
##### Solve system
#####

F̃ = fft(F, [1, 2])

btsolver = BatchedTridiagonalSolver(arch, dl=ld, d=d, du=ud, f=F̃, grid=fake_grid)

ϕ̃ = zeros(Complex{Float64}, Nx, Ny, Nz)
solve_batched_tridiagonal_system!(ϕ̃, arch, btsolver)

ϕ = CellField(Float64, arch, fake_grid)
interior(ϕ) .= real.(ifft(ϕ̃, [1, 2]))
# ϕ.data .= ϕ.data .- mean(interior(ϕ));

@show sum(interior(ϕ));

sum(interior(ϕ)) = -8294.993117803755


In [337]:
#####
##### Compute Laplacian of solution ϕ to test that it's correct
#####

# Gotta fill halo regions
pbcs = HorizontallyPeriodicBCs()
fill_halo_regions!(ϕ.data, pbcs, arch, fake_grid)

∇²ϕ = CellField(Float64, arch, fake_grid)

for i in 1:Nx, j in 1:Ny, k in 1:Nz
    ∇²ϕ.data[i, j, k] = ∇²(i, j, k, Δx, Δy, ΔzF, ΔzC, ϕ.data)
end

In [338]:
interior(∇²ϕ) .- R

8×8×8 Array{Float64,3}:
[:, :, 1] =
 -0.0519672   0.0428594    0.00492816  …   0.0035035   0.0315548 
 -0.0322063   0.0688201    0.0068719      -0.0357199   0.00758758
  0.0303958   0.029453    -0.0566804      -0.0614596  -0.00164644
  0.0537269   9.30846e-5   0.0467392       0.0128635  -0.0459885 
 -0.0250847   0.0124558   -0.00101794     -0.0164288   0.00811405
  0.0112877   0.026839    -0.0655989   …   0.0310817  -0.0535501 
 -0.0367668  -0.0137457    0.0170272      -0.0402252   0.00666418
  0.0525833  -0.0142669   -0.0495683       0.0644572  -0.019223  

[:, :, 2] =
 -1.4988e-15   -4.44089e-16  -1.4988e-15   …  -8.46545e-16   7.21645e-16
 -1.94289e-16  -1.94289e-15   6.66134e-16     -4.44089e-16  -1.05471e-15
  2.08167e-15  -4.27089e-15  -1.04083e-17      6.10623e-16  -3.55271e-15
 -4.85723e-16  -2.498e-15     1.52656e-16      1.63758e-15  -2.47025e-15
 -6.66134e-16   4.10783e-15   3.94129e-15      2.06779e-15   2.60902e-15
  4.0315e-15   -1.58207e-15   2.77556e-15  …   1.22125e-15